In [9]:
del(sc)

In [1]:
import pyspark

from pyspark import SparkConf, SparkContext,SQLContext

# Chapter 8 스파크 최적화 및 디버깅

## SparkConf로 스파크 설정하기

- 스파크의 최적화란 것은 종종 단순하게 스파크 애플리케이션의 실행 설정을 바꾸는 것만을 뜻하기도 함.

In [8]:
conf = SparkConf()
conf.set("spark.app.name", "My Spark App")
conf.set("spark.master", "local[4]")
conf.set("spark.ui.port", "36000")

sc = SparkContext(conf)

AttributeError: 'SparkConf' object has no attribute '_get_object_id'

In [7]:
# 예제 8-1 파이썬에서 SparkConf를 이용하여 애플리케이션 생성
conf = SparkConf()
conf.setAppName("My Spark App")
conf.setMaster("local[4]")
conf.setSparkHome("36000")

sc = SparkContext(conf)

AttributeError: 'SparkConf' object has no attribute '_get_object_id'

In [11]:
sc = SparkContext(master="local[4]", appName="My Spark App", sparkHome="36000")

- SparkConf 객체는 사용자가 재정의해서 쓸 수 있는 설정 옵션들에 대한 키와 값의 쌍들을 갖고 있음.
- 스파크는 spark-submit 도구를 써서 동적으로 설정값을 지정할 수 있음.
    - SparkConf가 만들어질 때 자동으로 감지되고 값을 채움
    - 내장된 가장 일반적인 스파크 설정값들에 대한 플래그와 함께 그 이외의 어느 스파크 설정값을 받아들일 수 있는 범용적인 --conf 플래그도 제공함.

In [ ]:
# 예제 8-4 플래그를 사용하여 실행 시의 설정값 지정
$ bin/spark-submit\
--class com.example.MyApp\
-- master local[4]\
-- name "My Spark App"\
-- conf spark.ui.port = 36000\
myApp.jar

- spark-submit은 또한 설정값을 파일에서 읽는 것도 지원함. -> 여러 사용자가 공유할 수 있는 환경 설정에 유용

In [ ]:
# 예제 8-5 기본값 파일을 사용한 실행 시의 설정값 지정
$ bin/spark-submit\
--class com.example.MyApp\
--properties-file my-config.conf
myApp.jar

## my-config.conf의 내용 ##
spark.master local[4]
spark.app.name "My Spark App"
spark.ui.port 36000

- 실행하는 애플리케이션의 SparkConf는 한 번 SparkContext의 생성자에 넘겨지고 나면 수정이 불가능하다. 즉 SparkContext가 초기화되기 전에 모든 설정값이 결정되어야 한다는 뜻이다.

- 동일한 설정 속성값이 여러 곳에 지정되는 경우
    - 정해진 우선 순위를 따름 : SparkConf 객체에 직접적으로 set()함수들을 호출 > spark-submit에 전달되는 플래그들 > 설정 파일 > 기본값

## 실행을 구성하는 것 : 작업, 태스크, 작업 단계

In [12]:
input = sc.textFile("input.txt")

In [19]:
# 단어 단위로 나누고 빈 라인 제거
tokenized = input.filter(lambda x: len(x) > 0).map(lambda x: x.split(" "))

In [20]:
tokenized.collect()

[['INFO', 'This', 'is', 'a', 'message', 'with', 'content'],
 ['INFO', 'This', 'is', 'some', 'other', 'content'],
 ['INFO', 'Here', 'are', 'more', 'messages'],
 ['WARN', 'This', 'is', 'a', 'warning'],
 ['ERROR', 'Something', 'bad', 'happened'],
 ['WARN', 'More', 'details', 'on', 'the', 'bad', 'thing'],
 ['INFO', 'back', 'to', 'normal', 'messages']]

In [21]:
# 각 라인의 헛 번째 단어(로그 레벨)을 추출하여 센다.
counts = tokenized.map(lambda x: (x[0], 1))

In [22]:
counts.collect()

[('INFO', 1),
 ('INFO', 1),
 ('INFO', 1),
 ('WARN', 1),
 ('ERROR', 1),
 ('WARN', 1),
 ('INFO', 1)]

In [23]:
counts = tokenized.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [24]:
counts.collect()

[('WARN', 2), ('INFO', 4), ('ERROR', 1)]

- counts는 각 로그 레벨 메시지의 개수를 갖게됨. 셸에서 이를 실행하고 나면 프로그램은 어떤 액션도 수행하지 않지만, 내부적으로 정의된 RDD 객체들의 지향성 비순환 그래프 (DAG, Directed Acycle Graph)를 갖게됨. 이것이 나중에 액션을 수행할 떄 쓰이게 된다.
- DAG : 수학이나 컴퓨터공학에서 순환하지않는(no directed cycles) 방향성(directed) 그래프 모델을 일컫는 말. 자료구조중 하나인 Graph 모델은 정보를 객체들 사이의 관계로 정의하고 있다.
- 각 RDD는 자신이 어떤 타입의 관계를 갖고 있는지에 대한 메타데이터에 따라 하나 이상의 부모 RDD를 가리키는 포인터를 유지함.

In [31]:
# toDebugString()으로 RDD 시각화
# 가계도

In [29]:
input.toDebugString()

b'(2) input.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  input.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [30]:
counts.toDebugString()

b'(2) PythonRDD[10] at collect at <ipython-input-24-124dfcab2f05>:1 []\n |  MapPartitionsRDD[9] at mapPartitions at PythonRDD.scala:374 []\n |  ShuffledRDD[8] at partitionBy at NativeMethodAccessorImpl.java:-2 []\n +-(2) PairwiseRDD[7] at reduceByKey at <ipython-input-23-d382fd708927>:1 []\n    |  PythonRDD[6] at reduceByKey at <ipython-input-23-d382fd708927>:1 []\n    |  input.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2 []\n    |  input.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:-2 []'

- 어떤 RDD가 만들어 졌는지 보여줌

### 가계도 제거가 일어나는 경우

- 파이프라이닝 외에도 스파크의 내부 스케줄러는 RDD가 이미 클러스터 메모리나 디스크에 캐싱되어 있는 경우 RDD그래프의 가계도를 제거할 수 있음.
    - 이 경우 앞부분은 생략하고 영속화되어 있는 RDD기존으로 연산을 시작한다.

- 이미 이전에 실행된 셔플링으로 인해, 심지어 persist()가 호출되지 않았음에도 일종의 부수 효과 (side effect)로 RDD의 데이터가 남아 있는 경우.
    - RDD 그래프가 여러 번 재연산되는 점을 이용하는 내부적 최적화의 결과임

In [32]:
# 예제 8-10 이미 캐싱된 RDD연산

# RDD 캐시
counts.cache()

# 첫 번떄 실행은 두 단계를 필요로 한다.
counts.collect()

# 두 번째 실행은 오직 한 단계만을 필요로 한다.
counts.collect()

[('WARN', 2), ('INFO', 4), ('ERROR', 1)]

-> UI로 확인 가능

- 특정 액션을 위해 생성되는 작업 단계(stage)들이 모여서 작업 (job)을 이룬다.
    - count()같은 액션을 호출할 때마다 하나 이상의 작업 단계로 구성된 job을 만드는 것.

- 물리 계획에서의 작업 단계들은 RDD 가계도에 따라 각자 의존성을 가지게되므로 그에 맞는 순서로 실행됨.

### 태스크 수행 순서
1. 저장 장치나 존재하는 RDD나 셔플 결과물로부터 입력 데이터를 가져온다.
2. RDD를 계싼해 내기 위해 필요한 연산들을 수행한다.
3. 결과를 셔플이나 외부 저장 장치에 쓰거나 드라이버에 되돌려 준다.

### 스파크 실행 중 발생하는 단계
1. 사용자 코드가 RDD의 DAG를 정의한다.
2. DAG가 액션의 실행계획으로 변환되게 한다.
3. 태스크들이 스케줄링되고 클러스터에서 실행된다.

## 스파크 웹 UI

- jobs : 진행 상황과 작업 단계, 태스크에 대한 수치들
- Storage : 영속화된 RDD의 정보
- Executors : 애플리케이션에 존재하는 익스큐터 목록
- Environment : 스파크 설정 디버깅

### 드라이버와 익스큐터 로그
- 드라이버 프로그램이나 익스큐터가 직접 생성한 로그 파일을 분석해서 스파크에 대한 많은 정보를 얻을 수 있음.
- 스파크 노드의 위치는 배포 모드에 따라 다름.
    - StandAlone
    - Mesos
    - Yarn
   

## 성능에 관한 핵심 고려 사항
- 병렬화 수준
- 직렬화 포멧
- 메모리 관리
- 하드웨어 프로비저닝

### 병렬화 수준
- 병렬화 개수가 너무 적으면 스파크가 리소스들을 놀리게 되는 경우 발생
- 병렬화가 너무 많다면 각 파티션에서의 작은 오버헤드라도 누적되면서 성능 문제가 심각해짐
- 병렬화 수준을 조정할 수 있는 두가지 방법
    - 데이터 셔플이 필요한 연산 간에 생성되는 RDD를 위한 병렬화 정도를 인자로 줄 수 있음.
    - 이미 존재하는 RDD를 더 적거나 더 많은 파티션을 갖도록 재배치할 수 있음.
        - repartition() : RDD를 무작위로 섞어 원하는 개수의 파티션으로 다시 나눠줌
        - coalesce() : 파티션 개수를 줄임.



In [ ]:
# 예제 8-11 PySpark 셸에서 클 RDD로 합치기
input = sc.textFile("s3:")
input.getNumPartitions()

lines = input.filter(lambda line : line.startwith("2014-10-17"))
lines.getNumPartitions()

lines = lines.coalesce(5).cache()
lines.getNumPartitions()

lines.count()

### 직렬화 포멧
- 스파크는 네트워크로 데이터를 전송하거나 디스크에 쓸 떄 객체들을 직렬화해 바이너리 포맷으로 변환시켜야함.
    - 기본적으로 자바에 내장된 직렬화를 이용함.
    - Kyro

- Kyro 직렬화 : spark.serializer 를 org.apache.spark.serializer.KyroSerializer로 지정해야함

In [36]:
# 카이로를 사용 설정하고 클래스 등록하기
conf = SparkConf()
conf.set("spark.serializer", "org.apache.spark.serializer.KyroSerializer")
conf.set("spark.kyro.registrationRequired", "true")
# conf.registerKyroClasses()

### 메모리 관리
- RDD 저장용
- 셔플 및 집합 연산 버퍼
- 사용자 코드

- RDD 저장용 60%, 셔플 메모리 20%, 사용자 프로그램 20%
    - 사용자 코드에서 매우 큰 객체를 할당하거나 한다면 저장용이나 셔플 메모리를 줄여서 메모리 고갈을 막을 수 있다.

** 기본 캐싱 정책 **
- MEMORY_ONLY : 새로운 RDD파티션을 저장할 메모리가 모자란다면 오래된 것들은 간단히 삭제 가능
- MEMORY_AND_DISK : 삭제 대신 RDD 파티션을 디스크에 내리게 되고 나중에 다시 필요해지면 간단하게 로컬 저장 장치에서 메모리로 읽어 들일 수 있게됨.
- MEMORY_ONLY_SER, MEMORY_AND_DISK_SER : 직렬화된 객체를 저장

### 하드웨어 프로비저닝
- 하드웨어 자원은 애플리케이션 완료 시간에 큰 영향을 끼침
    - 각 익스큐터의 메모리양, 코어 개수, 익스큐터의 총 개수, 로컬 디스크 개수 등
    
- 익스큐터가 쓰는 메모리 사이징은 많아도 좋지 않을 수 있음
    - 너무 큰 힙을 지정하는 것은 가비지 컬렉션에 의한 정지 현상이 길어져 스파크 작업의 전체 머리량에 심각한 피해를 끼칠 수 있음. 이 문제를 완화하기 위해 더 작은 익스큐터 메모리를 저장하는 것이 이득일 때도 있음.